# Create training dataset

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

# Functions

In [2]:
def get_subregion(dist1, dist2, place):
    row = dist1.loc[dist1['Place0']==place]
    shape = row['geometry'].to_list()
    shape = shape[0]

    places = dist2['geometry']
    res = places.within(shape)
    res = res.to_list()
    temp = dist2
    temp['res'] = res
    temp = temp.loc[temp['res']==True]
    return(temp)

### tweets

In [3]:
def plot_Tweets(dist, tweets, n):    
    Nu_Tweets = []
    for index, row in dist.iterrows():
        size = len(tweets.loc[tweets['Place' + str(n)]==row['Place'+ str(n)]])

        Nu_Tweets.append(size)

    Nu_Tweets
    dist['Tweets'] = Nu_Tweets

    plot = dist.explore(
        column="Tweets", # make choropleth based on "Data" column
        tooltip=['Place'+ str(n),'Tweets'], # show "Province, Data" value in tooltip (on hover)
        popup=True, # show all values in popup (on click)
        tiles="CartoDB positron", # use "CartoDB positron" tiles
        cmap="Paired", # https://matplotlib.org/stable/tutorials/colors/colormaps.html
        style_kwds=dict(color="black") # use black outline
        )
    return(plot)

In [4]:
def tweet_rate_av2(dates, tweets):
    tweet_day = [0]*10
    for i, date in enumerate(dates):
        day = date
        d1 = day - timedelta(days=5) 
        d2 = day + timedelta(days=5) 
        delta = (d2-d1).days
        for i in range(delta):
            day = d1+timedelta(days=i)
            NuTweets = len(tweets.loc[tweets['Date']==day]) 
            tweet_day[i]=tweet_day[i]+NuTweets

    # getting the average
    tweet_day =[x/len(dates) for x in tweet_day]

    # normalising
    #tweet_day =[x/max(tweet_day) for x in tweet_day]

    # Getting the average of the sets
    average1 = [sum(tweet_day)/len(tweet_day)]*10

    # this is just for the x axis
    date = list(range(1,10+1))


    plt.plot(date,tweet_day, "-r", label="Tweets")
    plt.plot(date,average1,"-b", label="Average Over All Set Dates")
    plt.axvline(x = 5, color = 'y', label = 'Centre (location of protest/non-protests)')
    plt.legend(loc="upper left")
    plt.ylabel('Number of Tweets')
    plt.xlabel('Date')
    plt.show()

In [5]:
def tweet_rate2(dates, tweets):
    for date in dates:
        day = date
        d1 = day - timedelta(days=5) 
        d2 = day + timedelta(days=5) 
        date2 = []
        tweet_day = []
        delta = (d2-d1).days

        for i in range(delta):
            day = d1+timedelta(days=i)
            NuTweets = len(tweets.loc[tweets['Date']==day]) 
            date2.append(i)
            tweet_day.append(NuTweets)

        # Normalising
        tweet_day =[x/max(tweet_day) for x in tweet_day]
        plt.plot(date2,tweet_day)
        plt.ylabel('Number of Tweets')
        plt.xlabel('Date') 

### Dates

In [6]:
def get_dates(protests,tweets):
    start = tweets.tail(1)['Date'].to_list()
    start = start[0]
    end = tweets.head(1)['Date'].to_list()
    end = end[0]

    delta = (end-start).days
    delta = delta
    yes = []
    no = []
    protests2 = []
    for i in range(delta):
        date = start + timedelta(days=i)
        temp = protests.loc[protests['Date']==date]
        if len(temp) == 0:
            no.append(date)
        else:
            yes.append(date)

    return(yes, no)

### Training set

In [7]:
def get_start(protests, tweets):
    dates = get_dates(protests, tweets)
    yes = dates[0]
    no = dates[1]

    res = [1]*len(yes)
    place = 'City of Cape Town'
    place = [place]*len(yes)
    data = {'Protest':res,'Date':yes,'Place':place}
    yes = pd.DataFrame(data)

    res = [0]*len(no)
    place = 'City of Cape Town'
    place = [place]*len(no)
    data = {'Protest':res,'Date':no,'Place':place}
    no = pd.DataFrame(data)

    # randomly sampling len(yes) samples from no such that there is an equal number of 
    # protests to non protest
    no = no.sample(len(yes))

    # Joining the dataframes, resetting the index and randomising the rows. 
    data = pd.concat([yes,no])
    data = data.sample(frac=0.7)
    data = data.reset_index(drop=True)

    return(data)

In [8]:
# return(tweet_day,average,specifiedTweets,length)
# return(likes,followers,retweets,replies,Subjectivity,Polarity)
# return(centre)

In [9]:
def get_tweets(date, tweets):
    day = date
    d1 = day - timedelta(days=5) 
    d2 = day + timedelta(days=5) 
    delta = (d2-d1).days
    delta = int(delta)
    tweet_day = [0]*(int(5*2))
    for i in range(10):
        day = d1+timedelta(days=i)
        NuTweets = len(tweets.loc[tweets['Date']==day]) 
        tweet_day[i]=tweet_day[i]+NuTweets
    # average 
    average = sum(tweet_day)/len(tweet_day)
    # normalising
    tweet_day =[x/max(tweet_day) for x in tweet_day]
    # # average 
    # average = sum(tweet_day)/len(tweet_day)
    # specific tweets.
    specifiedTweets = tweets[((tweets['Date'] >d1)&(tweets['Date'] <d2))]
    specifiedTweets = specifiedTweets.reset_index(drop=True)
    length = len(specifiedTweets)

    return(tweet_day,average,specifiedTweets,length)

In [10]:
def tweet_metrics(tweets):
    length = len(tweets)
    followers = (tweets['author_followers'].sum())/length
    retweets = (tweets['retweets'].sum())/length
    replies = (tweets['replies'].sum())/length
    likes = (tweets['likes'].sum())/length
    Subjectivity = (tweets['Subjectivity'].sum())/length
    Polarity = (tweets['Polarity'].sum())/length


    return(likes,followers,retweets,replies,Subjectivity,Polarity)

In [11]:
def get_centre(tweets):
    # locations = tweets['geometry'].centroid
    # print(locations.head())
    # locations = locations.reset_index(drop=True)
    # centre = locations.centriod

    # its not working for now
    centre = 'too bad'
    return(centre)

# Training Data

In [12]:
def get_training(data, tweets):
    NuTweets = []
    average = []
    d1 = []
    d2 = []
    d3 = []
    d4 = []
    d5 = []
    d6 = []
    d7 = []
    d8 = []
    d9 = []
    d10 = []

    likes = []
    followers = []
    retweets = []
    replies = []
    sub = []
    pol = []

    centre = []


    for index, row in data.iterrows():
        date = row['Date']
        
        res1 = get_tweets(date,tweets)
        res2 = tweet_metrics(res1[2])
        res3 = get_centre(res1[2])


        average.append(res1[1])
        NuTweets.append(res1[3])
        d1.append(res1[0][0])
        d2.append(res1[0][1])
        d3.append(res1[0][2])
        d4.append(res1[0][3])
        d5.append(res1[0][4])
        d6.append(res1[0][5])
        d7.append(res1[0][6])
        d8.append(res1[0][7])
        d9.append(res1[0][8])
        d10.append(res1[0][9])


        likes.append(res2[0])
        followers.append(res2[1])
        retweets.append(res2[2])
        replies.append(res2[3])
        sub.append(res2[4])
        pol.append(res2[5])

        centre.append(res3)

    

    metrics = {'NuTweets':NuTweets,'d1':d1,'d2':d2,'d3':d3,'d4':d4,'d5':d5,'d6':d6,'d7':d7,'d8':d8,'d9':d9,'d10':d10,'average':average,'likes':likes,'followers':followers,'retweets':retweets,'replies':replies,'sub':sub,'pol':pol}
    metrics = pd.DataFrame(metrics)

        
    training = pd.concat([data,metrics.reindex(data.index)], axis=1)
    training = training.drop(['Date','Place'], axis = 1)
    return(training)

# MAIN

### Downloading tweets and protests

In [29]:
tweets = pd.read_csv('DATA/tweets.csv')
tweets['geometry'] = gpd.GeoSeries.from_wkt(tweets['geometry'])
tweets['Date']=pd.to_datetime(tweets['Date'], format='%Y %m %d')
tweets = tweets.drop(['Unnamed: 0','Analysis'],axis=1)

In [30]:
protests = pd.read_csv('DATA/protests.csv')
protests['geometry'] = gpd.GeoSeries.from_wkt(protests['geometry'])
protests['Date']=pd.to_datetime(protests['Date'], format='%Y %m %d')
protests = protests.drop(['Unnamed: 0'],axis=1)

In [31]:
tweets.head(1)

,text,Date,geometry,author_followers,retweets,replies,likes,quote_count,Subjectivity,Polarity,Place0,Place1,Place2,Place3
0,@unisa the 3rd January registration is it for ...,2017-12-31,POINT (30.64532 -29.81931),1100,0,0,0,0,0.7,0.0,KwaZulu-Natal,eThekwini,eThekwini,6


In [32]:
protests.head(1)

,Date,notes,geometry,Place0,Place1,Place2,Place3
0,2017-10-25,A few hundred Fees Must Fall protesters attemp...,POINT (18.47640 -33.96330),Western Cape,City of Cape Town,City of Cape Town,58


# Lets test protests in the WesternCape

In [33]:
protests = protests.loc[protests['Place0']=='Western Cape']
tweets = tweets.loc[tweets['Place0']=='Western Cape']

In [34]:
data = get_start(protests,tweets)
training = get_training(data,tweets)
training.head(2)

,Protest,NuTweets,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,average,likes,followers,retweets,replies,sub,pol
0,1,16,0.333333,0.0,0.666667,0.00,0.00,1.00,1.0,1.0,1.0,0.666667,1.7,2.062500,3224.125000,0.625000,0.375000,0.306182,0.039183
1,0,19,0.750000,1.0,0.000000,0.75,0.75,0.25,0.5,0.5,1.0,0.000000,2.2,0.947368,20327.157895,0.578947,0.263158,0.426557,0.155142


# Creating a simple model

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [36]:
# test train split
from sklearn.linear_model import LogisticRegression


X_train,X_test,y_train,y_test = train_test_split(training.drop('Protest',axis = 1), training['Protest'],test_size = 0.3,random_state=7)
# training the model
logreg =  LogisticRegression(solver='lbfgs')
logreg.fit(X_train,y_train)

c:\Users\Joseph\anaconda3\envs\geo-env\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [37]:
logreg.score(X_test,y_test)

0.7857142857142857